In [1]:
# 필요한 모듈들
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline   
import xlrd
import wbgapi as wb
from gedaTools import timeToInt

In [ ]:
totalDT = wb.data.DataFrame\
    (['SP.DYN.CONU.ZS', 'SG.GEN.PARL.ZS', 'SL.AGR.EMPL.FE.ZS', 'SL.IND.EMPL.FE.ZS', 'SL.SRV.EMPL.FE.ZS'],\
        skipBlanks=True, columns='series')

In [ ]:
totDDT = totalDT.dropna(how='any')

In [ ]:
totRST = totDDT.reset_index()

In [ ]:
totRST

In [ ]:
totH = totRST.iloc[:, 2:]
totH

In [ ]:
plt.figure(figsize=(7, 5))
sns.set(style="whitegrid")
ax=sns.scatterplot(x='SP.DYN.CONU.ZS', y='SG.GEN.PARL.ZS', data=totH, linewidth=2.5)

plt.figure(figsize=(7, 5))
sns.set(style="whitegrid")
ax=sns.scatterplot(x='SP.DYN.CONU.ZS', y='SL.AGR.EMPL.FE.ZS', data=totH, linewidth=2.5)

plt.figure(figsize=(7, 5))
sns.set(style="whitegrid")
ax=sns.scatterplot(x='SP.DYN.CONU.ZS', y='SL.IND.EMPL.FE.ZS', data=totH, linewidth=2.5)

plt.figure(figsize=(7, 5))
sns.set(style="whitegrid")
ax=sns.scatterplot(x='SP.DYN.CONU.ZS', y='SL.SRV.EMPL.FE.ZS', data=totH, linewidth=2.5)

# 직군별로 다른 모습

In [ ]:
conPIV = pd.pivot_table(totH, index='SP.DYN.CONU.ZS')
conPIV

In [ ]:
fig, axes = plt.subplots(figsize=(30, 30))
sns.heatmap(conPIV, cmap='RdBu', ax=axes)
plt.xlabel('Female Employment')
plt.ylabel('Female Contraceptive prevalence')
plt.show()